In [1]:
import numpy as np
import sys
import os
import pandas as pd

In [34]:
column_names = ["name","red","green","blue","alpha","id"]

In [2]:
class SUNRBDBase:
    SPLITS = ['train', 'test']

    # number of classes without void
    N_CLASSES = 37

    CLASS_NAMES_ENGLISH = ['void', 'wall', 'floor', 'cabinet', 'bed', 'chair',
                           'sofa', 'table', 'door', 'window', 'bookshelf',
                           'picture', 'counter', 'blinds', 'desk', 'shelves',
                           'curtain', 'dresser', 'pillow', 'mirror',
                           'floor mat', 'clothes', 'ceiling', 'books',
                           'fridge', 'tv', 'paper', 'towel', 'shower curtain',
                           'box', 'whiteboard', 'person', 'night stand',
                           'toilet', 'sink', 'lamp', 'bathtub', 'bag']

    CLASS_NAMES_GERMAN = ['Void', 'Wand', 'Boden', 'Schrank', 'Bett', 'Stuhl',
                          'Sofa', 'Tisch', 'Tür', 'Fenster', 'Bücherregal',
                          'Bild', 'Tresen', 'Jalousien', 'Schreibtisch',
                          'Regal', 'Vorhang', 'Kommode', 'Kissen', 'Spiegel',
                          'Bodenmatte', 'Kleidung', 'Zimmerdecke', 'Bücher',
                          'Kühlschrank', 'Fernseher', 'Papier', 'Handtuch',
                          'Duschvorhang', 'Kiste', 'Whiteboard', 'Person',
                          'Nachttisch', 'Toilette', 'Waschbecken', 'Lampe',
                          'Badewanne', 'Tasche']

    CLASS_COLORS = [(0, 0, 0), (119, 119, 119), (244, 243, 131),
                    (137, 28, 157), (150, 255, 255), (54, 114, 113),
                    (0, 0, 176), (255, 69, 0), (87, 112, 255), (0, 163, 33),
                    (255, 150, 255), (255, 180, 10), (101, 70, 86),
                    (38, 230, 0), (255, 120, 70), (117, 41, 121),
                    (150, 255, 0), (132, 0, 255), (24, 209, 255),
                    (191, 130, 35), (219, 200, 109), (154, 62, 86),
                    (255, 190, 190), (255, 0, 255), (192, 79, 212),
                    (152, 163, 55), (230, 230, 230), (53, 130, 64),
                    (155, 249, 152), (87, 64, 34), (214, 209, 175),
                    (170, 0, 59), (255, 0, 0), (193, 195, 234), (70, 72, 115),
                    (255, 255, 0), (52, 57, 131), (12, 83, 45)]

In [3]:
def _get_colormap(n):
    def bitget(byteval, idx):
        return (byteval & (1 << idx)) != 0

    cmap = np.zeros((n, 3), dtype='uint8')
    for i in range(n):
        r = g = b = 0
        c = i
        for j in range(8):
            r = r | (bitget(c, 0) << 7-j)
            g = g | (bitget(c, 1) << 7-j)
            b = b | (bitget(c, 2) << 7-j)
            c = c >> 3

        cmap[i] = np.array([r, g, b])

    return cmap


In [4]:
class NYUv2Base:
    SPLITS = ['train', 'test']
    SPLIT_FILELIST_FILENAMES = {SPLITS[0]: 'train.txt', SPLITS[1]: 'test.txt'}
    SPLIT_DIRS = {SPLITS[0]: 'train', SPLITS[1]: 'test'}

    # number of classes without void
    N_CLASSES = [894, 40, 13]

    DEPTH_DIR = 'depth'
    DEPTH_RAW_DIR = 'depth_raw'
    RGB_DIR = 'rgb'

    LABELS_DIR_FMT = 'labels_{:d}'
    LABELS_COLORED_DIR_FMT = 'labels_{:d}_colored'

    CLASS_NAMES_13 = ['void',
                      'bed', 'books', 'ceiling', 'chair', 'floor', 'furniture',
                      'objects', 'picture', 'sofa', 'table', 'tv', 'wall',
                      'window']
    CLASS_NAMES_40 = ['void',
                      'wall', 'floor', 'cabinet', 'bed', 'chair', 'sofa',
                      'table', 'door', 'window', 'bookshelf', 'picture',
                      'counter', 'blinds', 'desk', 'shelves', 'curtain',
                      'dresser', 'pillow', 'mirror', 'floor mat', 'clothes',
                      'ceiling', 'books', 'refridgerator', 'television',
                      'paper', 'towel', 'shower curtain', 'box', 'whiteboard',
                      'person', 'night stand', 'toilet', 'sink', 'lamp',
                      'bathtub', 'bag',
                      'otherstructure', 'otherfurniture', 'otherprop']
    
    CLASS_COLORS_13 = [[0, 0, 0],
                       [0, 0, 255],
                       [232, 88, 47],
                       [0, 217, 0],
                       [148, 0, 240],
                       [222, 241, 23],
                       [255, 205, 205],
                       [0, 223, 228],
                       [106, 135, 204],
                       [116, 28, 41],
                       [240, 35, 235],
                       [0, 166, 156],
                       [249, 139, 0],
                       [225, 228, 194]]

    CLASS_COLORS_40 = _get_colormap(1+40).tolist()

### NYU_V2 13 Class

In [40]:
num_classes = 13
alphas = np.ones((num_classes + 1 ,1), dtype=np.int) * 255
colors = np.array(NYUv2Base.CLASS_COLORS_13)
names =  np.array(NYUv2Base.CLASS_NAMES_13).reshape((num_classes + 1 ,1))
indexes = np.arange(num_classes + 1 ).reshape((num_classes + 1 ,1))

data = np.concatenate((names, colors, alphas, indexes), axis=1)
print(data.shape)

pd_nyu_13 = pd.DataFrame(data=data, columns=column_names)
pd_nyu_13.to_csv ('nyu_13_segmentation_mapping.csv', index = False, header=True)
pd_nyu_13.head()

(14, 6)


,name,red,green,blue,alpha,id
0,void,0,0,0,255,0
1,bed,0,0,255,255,1
2,books,232,88,47,255,2
3,ceiling,0,217,0,255,3
4,chair,148,0,240,255,4


### NYU_V2 40 Class

In [41]:
num_classes = 40
alphas = np.ones((num_classes + 1 ,1), dtype=np.int) * 255
colors = np.array(NYUv2Base.CLASS_COLORS_40)
names =  np.array(NYUv2Base.CLASS_NAMES_40).reshape((num_classes + 1 ,1))
indexes = np.arange(num_classes + 1 ).reshape((num_classes + 1 ,1))

data = np.concatenate((names, colors, alphas, indexes), axis=1)
print(data.shape)

pd_nyu_40 = pd.DataFrame(data=data, columns=column_names)
pd_nyu_40.to_csv ('nyu_40_segmentation_mapping.csv', index = False, header=True)
pd_nyu_40.head()

(41, 6)


,name,red,green,blue,alpha,id
0,void,0,0,0,255,0
1,wall,128,0,0,255,1
2,floor,0,128,0,255,2
3,cabinet,128,128,0,255,3
4,bed,0,0,128,255,4


### SUNRGBD 37 Class

In [42]:
num_classes = 37
alphas = np.ones((num_classes + 1 ,1), dtype=np.int) * 255
colors = np.array(SUNRBDBase.CLASS_COLORS)
names =  np.array(SUNRBDBase.CLASS_NAMES_ENGLISH).reshape((num_classes + 1 ,1))
indexes = np.arange(num_classes + 1 ).reshape((num_classes + 1 ,1))

data = np.concatenate((names, colors, alphas, indexes), axis=1)
print(data.shape)

pd_sunrgbd_37 = pd.DataFrame(data=data, columns=column_names)
pd_sunrgbd_37.to_csv ('sunrgbd_37_segmentation_mapping.csv', index = False, header=True)
pd_sunrgbd_37.head()

(38, 6)


,name,red,green,blue,alpha,id
0,void,0,0,0,255,0
1,wall,119,119,119,255,1
2,floor,244,243,131,255,2
3,cabinet,137,28,157,255,3
4,bed,150,255,255,255,4
